# RDKit Molecular Descriptors Experiment

**CRITICAL INSIGHT**: ALL model families follow the SAME CV-LB relationship.
- LB = 4.23×CV + 0.0533 (R²=0.981)
- Intercept (0.0533) > Target (0.0347) → Current approach CANNOT reach target
- This suggests the problem is in the FEATURES, not the models

**Hypothesis**: RDKit provides 200+ molecular descriptors that might capture different aspects of solvent chemistry than Spange (13 features).

**Why this might work**:
- RDKit descriptors include: MolWt, LogP, TPSA, HBD, HBA, rotatable bonds, etc.
- These are fundamentally different from Spange descriptors
- Might capture different aspects of solvent-solute interactions
- Could have a DIFFERENT CV-LB relationship

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load SMILES lookup and compute RDKit descriptors
SMILES_DF = pd.read_csv(f'{DATA_PATH}/smiles_lookup.csv', index_col=0)
print(f'SMILES lookup: {SMILES_DF.shape}')
print(f'Solvents: {SMILES_DF.index.tolist()[:10]}...')

# Get all descriptor names
desc_names = [name for name, _ in Descriptors._descList]
print(f'Number of RDKit descriptors: {len(desc_names)}')

# Create calculator
calc = MolecularDescriptorCalculator(desc_names)

# Calculate descriptors for each solvent
def get_rdkit_descriptors(smiles):
    """Calculate RDKit descriptors for a SMILES string."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [np.nan] * len(desc_names)
    return list(calc.CalcDescriptors(mol))

# Create lookup table for RDKit descriptors
rdkit_features = {}
for solvent in SMILES_DF.index:
    smiles = SMILES_DF.loc[solvent, 'solvent smiles']
    rdkit_features[solvent] = get_rdkit_descriptors(smiles)

# Convert to DataFrame
RDKIT_DF = pd.DataFrame(rdkit_features, index=desc_names).T
print(f'RDKit descriptors shape: {RDKIT_DF.shape}')

# Remove columns with NaN or constant values
RDKIT_DF = RDKIT_DF.dropna(axis=1)
RDKIT_DF = RDKIT_DF.loc[:, RDKIT_DF.var() > 0]
print(f'RDKit descriptors after filtering: {RDKIT_DF.shape}')
print(f'Sample descriptors: {RDKIT_DF.columns.tolist()[:10]}')

SMILES lookup: (26, 1)
Solvents: ['Cyclohexane', 'Ethyl Acetate', 'Acetic Acid', '2-Methyltetrahydrofuran [2-MeTHF]', '1,1,1,3,3,3-Hexafluoropropan-2-ol', 'IPA [Propan-2-ol]', 'Ethanol', 'Methanol', 'Ethylene Glycol [1,2-Ethanediol]', 'Acetonitrile']...
Number of RDKit descriptors: 217
RDKit descriptors shape: (26, 217)
RDKit descriptors after filtering: (26, 133)
Sample descriptors: ['MaxAbsEStateIndex', 'MaxEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed', 'SPS', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons']


In [4]:
# Check which solvents are in training data
X_single, Y_single = load_data("single_solvent")
training_solvents = X_single["SOLVENT NAME"].unique()
print(f'Training solvents: {len(training_solvents)}')
print(f'RDKit solvents: {len(RDKIT_DF.index)}')

# Check overlap
overlap = set(training_solvents) & set(RDKIT_DF.index)
print(f'Overlap: {len(overlap)}')

# Check missing solvents
missing = set(training_solvents) - set(RDKIT_DF.index)
if missing:
    print(f'Missing solvents: {missing}')

Training solvents: 24
RDKit solvents: 26
Overlap: 24


In [5]:
# RDKit Featurizer
class RDKitFeaturizer:
    """Featurizer using RDKit molecular descriptors.
    
    Uses ~200 RDKit descriptors instead of Spange (13 features).
    """
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.rdkit_df = RDKIT_DF
        # Features: 2 (time, temp) + 3 (kinetics) + n_rdkit
        self.feats_dim = 2 + 3 + self.rdkit_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_rdkit = self.rdkit_df.loc[X["SOLVENT A NAME"]].values
            B_rdkit = self.rdkit_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_rdkit = B_rdkit * (1 - (1-pct)) + A_rdkit * (1-pct)
            else:
                X_rdkit = A_rdkit * (1 - pct) + B_rdkit * pct
        else:
            X_rdkit = self.rdkit_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_rdkit])

print(f'RDKit feature dimension: {RDKitFeaturizer().feats_dim}')

RDKit feature dimension: 138


In [6]:
# Simple Featurizer for GP (use subset of RDKit descriptors)
class SimpleRDKitFeaturizer:
    """Simpler featurizer for GP using top RDKit descriptors."""
    def __init__(self, mixed=False, n_features=20):
        self.mixed = mixed
        # Select top n_features by variance
        variances = RDKIT_DF.var().sort_values(ascending=False)
        self.top_features = variances.head(n_features).index.tolist()
        self.rdkit_df = RDKIT_DF[self.top_features]
        self.feats_dim = 2 + 3 + len(self.top_features)

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_rdkit = self.rdkit_df.loc[X["SOLVENT A NAME"]].values
            B_rdkit = self.rdkit_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_rdkit = B_rdkit * (1 - (1-pct)) + A_rdkit * (1-pct)
            else:
                X_rdkit = A_rdkit * (1 - pct) + B_rdkit * pct
        else:
            X_rdkit = self.rdkit_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_rdkit])

print(f'Simple RDKit feature dimension (for GP): {SimpleRDKitFeaturizer().feats_dim}')

Simple RDKit feature dimension (for GP): 25


In [7]:
# Enhanced MLP with residual connections
class EnhancedMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=3, dropout=0.2):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.hidden1 = nn.Linear(hidden_dim, hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim)
        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.ln3 = nn.LayerNorm(hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.act = nn.GELU()
        
    def forward(self, x):
        x = self.act(self.ln1(self.input_proj(x)))
        x = self.dropout(x)
        residual = x
        x = self.act(self.ln2(self.hidden1(x)))
        x = self.dropout(x)
        x = x + residual
        residual = x
        x = self.act(self.ln3(self.hidden2(x)))
        x = self.dropout(x)
        x = x + residual
        return self.output(x)

print('EnhancedMLP defined')

EnhancedMLP defined


In [8]:
# GP + MLP + LGBM Ensemble with RDKit Features
class RDKitEnsemble:
    """Ensemble using RDKit molecular descriptors.
    
    Uses the best ensemble weights from exp_035: GP(0.15) + MLP(0.55) + LGBM(0.3)
    But with RDKit descriptors instead of Spange + DRFP + ACS PCA.
    """
    def __init__(self, data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.3):
        self.data_type = data
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        
        # Featurizers
        self.full_featurizer = RDKitFeaturizer(mixed=(data == 'full'))
        self.simple_featurizer = SimpleRDKitFeaturizer(mixed=(data == 'full'), n_features=20)
        
        # Scalers
        self.full_scaler = StandardScaler()
        self.simple_scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        
        # Models
        self.mlp = EnhancedMLP(self.full_featurizer.feats_dim, hidden_dim=128, output_dim=3, dropout=0.2).to(device)
        self.gps = []
        self.lgbm_models = []
        
    def train_model(self, X, Y):
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        # Scale
        X_full_scaled = self.full_scaler.fit_transform(X_full)
        X_simple_scaled = self.simple_scaler.fit_transform(X_simple)
        Y_scaled = self.y_scaler.fit_transform(Y_np)
        
        # Train MLP
        self._train_mlp(X_full_scaled, Y_scaled)
        
        # Train GPs (one per target)
        self.gps = []
        for i in range(3):
            kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=3, normalize_y=True, random_state=42)
            gp.fit(X_simple_scaled, Y_scaled[:, i])
            self.gps.append(gp)
        
        # Train LGBM (one per target)
        self.lgbm_models = []
        for i in range(3):
            lgbm_model = lgb.LGBMRegressor(
                n_estimators=200, learning_rate=0.05, max_depth=6,
                num_leaves=31, min_child_samples=10, reg_alpha=0.1, reg_lambda=0.1,
                random_state=42, verbose=-1
            )
            lgbm_model.fit(X_full_scaled, Y_scaled[:, i])
            self.lgbm_models.append(lgbm_model)
    
    def _train_mlp(self, X_scaled, Y_scaled):
        X_tensor = torch.tensor(X_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_scaled, dtype=torch.double).to(device)
        
        dataset = TensorDataset(X_tensor, Y_tensor)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        
        optimizer = torch.optim.AdamW(self.mlp.parameters(), lr=1e-3, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=150)
        
        target_weights = torch.tensor([1.0, 1.0, 2.0], dtype=torch.double).to(device)
        
        self.mlp.train()
        for epoch in range(150):
            for X_batch, Y_batch in loader:
                optimizer.zero_grad()
                pred = self.mlp(X_batch)
                loss = ((pred - Y_batch) ** 2 * target_weights).mean()
                loss.backward()
                optimizer.step()
            scheduler.step()
    
    def predict(self, X):
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        # Scale
        X_full_scaled = self.full_scaler.transform(X_full)
        X_simple_scaled = self.simple_scaler.transform(X_simple)
        
        # MLP prediction
        self.mlp.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
            mlp_pred_scaled = self.mlp(X_tensor).cpu().numpy()
        
        # GP prediction
        gp_pred_scaled = np.column_stack([gp.predict(X_simple_scaled) for gp in self.gps])
        
        # LGBM prediction
        lgbm_pred_scaled = np.column_stack([lgbm.predict(X_full_scaled) for lgbm in self.lgbm_models])
        
        # Ensemble
        ensemble_pred_scaled = (
            self.gp_weight * gp_pred_scaled + 
            self.mlp_weight * mlp_pred_scaled + 
            self.lgbm_weight * lgbm_pred_scaled
        )
        
        # Inverse transform
        ensemble_pred = self.y_scaler.inverse_transform(ensemble_pred_scaled)
        ensemble_pred = np.clip(ensemble_pred, 0, 1)
        
        return torch.tensor(ensemble_pred)
    
    def predict_with_tta(self, X):
        if self.data_type == 'single':
            return self.predict(X)
        
        pred1 = self.predict(X)
        
        X_flip = X.copy()
        X_flip["SOLVENT A NAME"] = X["SOLVENT B NAME"]
        X_flip["SOLVENT B NAME"] = X["SOLVENT A NAME"]
        X_flip["SolventB%"] = 1 - X["SolventB%"]
        
        pred2 = self.predict(X_flip)
        
        return (pred1 + pred2) / 2

print('RDKitEnsemble defined with GP(0.15) + MLP(0.55) + LGBM(0.3) + RDKit Features')

RDKitEnsemble defined with GP(0.15) + MLP(0.55) + LGBM(0.3) + RDKit Features


In [9]:
# Test the model
X_single, Y_single = load_data("single_solvent")
print(f"Single solvent data: {X_single.shape[0]} samples, {len(X_single['SOLVENT NAME'].unique())} unique solvents")

X_full, Y_full = load_data("full")
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
print(f"Full data: {X_full.shape[0]} samples, {len(ramps)} unique ramps")

Single solvent data: 656 samples, 24 unique solvents
Full data: 1227 samples, 13 unique ramps


In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = RDKitEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:23, 23.80s/it]

2it [00:49, 24.71s/it]

3it [01:08, 22.37s/it]

4it [01:29, 21.58s/it]

...

21it [09:24, 28.37s/it]

22it [09:51, 27.81s/it]

23it [10:20, 28.35s/it]

24it [10:50, 28.60s/it]

24it [10:50, 27.08s/it]

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = RDKitEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict_with_tta(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== RDKIT DESCRIPTORS CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')

print(f'\n=== COMPARISON ===')
print(f'exp_032 (best CV, Spange + DRFP + ACS PCA): CV 0.008194')
print(f'exp_048 (RDKIT DESCRIPTORS): CV {overall_mse:.6f}')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_032!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_032')
    
# Estimate LB using old relationship
estimated_lb = 4.23 * overall_mse + 0.0533
print(f'\nEstimated LB (using old relationship): {estimated_lb:.4f}')
print(f'Best LB so far: 0.0877')
print(f'Target: 0.0347')
print(f'\nKey question: Do RDKit descriptors have a DIFFERENT CV-LB relationship?')
print(f'If they have lower intercept, we might be able to reach the target.')